# Direct and inverse geometry of 2d robots



## Displaying objects

Let's first learn how to open a 3D viewer, in which we will build our simulator. First start gepetto-gui. Best is to run it directly from the shell by typing gepetto-gui. A new window with the Gepetto logo should open. Objects can be now created from the python commands.

The following GView object is a client of the Gepetto Viewer server, i.e. it will be use to pass display command to the viewer. The first commands are to create objects, 

In [1]:
import gviewserver
gv = gviewserver.GepettoViewerServer()

gv.addSphere('world/ball', .1, [1, 0, 0, 1])  # radius, color=[r,g,b,1]
gv.addCapsule('world/capsule', .05,.75, [1, 1, 1, 1])  # radius, length, color = [r,g,b,a]
gv.addBox('world/box', .2,.05,.5,  [.5, .5, 1, 1])  # depth(x), length(y), height(z), color

True

Execute the above python commands once, you get a "True" output. Execute it a second time, you get a False: that's just telling you that the object world/box already exists and Gepetto viewer cannot create it again. If you want to erase your world and all your objects, just run:


In [2]:
gv.deleteNode('world', True)  # name, all=True

You now have to run again the gviewerserver.GepettoViewerServer() command to create the world again.


In [3]:
gv = gviewserver.GepettoViewerServer()

Placing objects can be done using the applyConfiguration command, and specifying the placement as a 3D translation and quaternion rotation. Don't forget to refresh your window after placing your objects.

In [4]:
gv.applyConfiguration('world/box', [.1, .1, .1, 1, 0, 0, 0])  # x, y, z, quaternion
gv.refresh()

In a first time, we will work in 2D. Here is a shortcut to place an object from x,y,theta 2d placement.

In [5]:
import numpy as np
def placement(x, y, theta): 
    return [y, 0, x, 0, np.sin(theta / 2), 0, np.cos(theta / 2)]
gv.applyConfiguration('world/capsule', placement(0.1, 0.2, np.pi / 4))
gv.refresh()

## Creating a 2d robot
This robot will have 2 joints, named shoulder and elbow, with link of length 1 to connect them. First let's create the 5 geometry objects.

In [6]:
gv.addSphere ('world/joint1', .1, [1 ,0 ,0,1])
gv.addSphere ('world/joint2', .1, [1 ,0 ,0,1])
gv.addSphere ('world/joint3', .1, [1 ,0 ,0,1])
gv.addCapsule('world/arm1', .05, .75, [1 ,1 ,1,1])
gv.addCapsule('world/arm2', .05, .75, [1 ,1 ,1,1])

True

Given a configuration vector q of dimension 2, compute the position of the centers of each object, and display correctly the robot.

In [7]:
q = np.matrix(np.random.rand(2) * 6 - 3).T

In [8]:
# %load tp1/solution_display_2r.py
import numpy as np


def placement(x, y, theta):
    return [y, 0, x, 0, np.sin(theta / 2), 0, np.cos(theta / 2)]


def solution_display_2r(q, gv):
    c0 = np.cos(q[0, 0])
    s0 = np.sin(q[0, 0])
    c1 = np.cos(q[0, 0] + q[1, 0])
    s1 = np.sin(q[0, 0] + q[1, 0])

    gv.applyConfiguration('world/joint1', placement(0, 0, 0))
    gv.applyConfiguration('world/arm1', placement(c0 / 2, s0 / 2, q[0, 0]))

    gv.applyConfiguration('world/joint2', placement(c0, s0, q[0, 0]))
    gv.applyConfiguration('world/arm2', placement(c0 + c1 / 2, s0 + s1 / 2, q[0, 0] + q[1, 0]))

    gv.applyConfiguration('world/joint3', placement(c0 + c1, s0 + s1, q[0, 0] + q[1, 0]))

    gv.refresh()


## Optimize the configuration 
Scipy is a collection of scientific tools for Python. It contains, in particular, a set of optimizers that we are going to use for solving the inverse-geometry problem. If not done yet, install scipy with `sudo apt-get install python3-scipy`

In [10]:
# %load tp1/example_scipy.py
'''
Example of use a the optimization toolbox of SciPy.
The function optimized here are meaningless, and just given
as example. They ***are not*** related to the robotic models.
'''
import numpy as np
from scipy.optimize import fmin_bfgs, fmin_slsqp


def cost(x):
    '''Cost f(x,y) = x^2 + 2y^2 - 2xy - 2x '''
    x0 = x[0]
    x1 = x[1]
    return -1 * (2 * x0 * x1 + 2 * x0 - x0**2 - 2 * x1**2)


def constraint_eq(x):
    ''' Constraint x^3 = y '''
    return np.array([x[0]**3 - x[1]])


def constraint_ineq(x):
    '''Constraint x>=2, y>=2'''
    return np.array([x[0] - 2, x[1] - 2])


class CallbackLogger:
    def __init__(self):
        self.nfeval = 1

    def __call__(self, x):
        print('===CBK=== {0:4d}   {1: 3.6f}   {2: 3.6f}'.format(self.nfeval, x[0], x[1], cost(x)))
        self.nfeval += 1


x0 = np.array([0.0, 0.0])
# Optimize cost without any constraints in BFGS, with traces.
xopt_bfgs = fmin_bfgs(cost, x0, callback=CallbackLogger())
print('\n *** Xopt in BFGS = %s \n\n\n\n' % str(xopt_bfgs))

# Optimize cost without any constraints in CLSQ
xopt_lsq = fmin_slsqp(cost, [-1.0, 1.0], iprint=2, full_output=1)
print('\n *** Xopt in LSQ = %s \n\n\n\n' % str(xopt_lsq))

# Optimize cost with equality and inequality constraints in CLSQ
xopt_clsq = fmin_slsqp(cost, [-1.0, 1.0], f_eqcons=constraint_eq, f_ieqcons=constraint_ineq, iprint=2, full_output=1)
print('\n *** Xopt in c-lsq = %s \n\n\n\n' % str(xopt_clsq))


===CBK===    1    1.000000   -0.000000
===CBK===    2    2.010000    1.010000
===CBK===    3    2.000000    1.000000
Optimization terminated successfully.
         Current function value: -2.000000
         Iterations: 3
         Function evaluations: 20
         Gradient evaluations: 5

 *** Xopt in BFGS = [2.00000009 1.00000005] 




  NIT    FC           OBJFUN            GNORM
    1     4     7.000000E+00     8.485281E+00
    2     9    -2.000000E-01     1.697056E+00
    3    13    -1.928000E+00     3.394112E-01
    4    17    -2.000000E+00     6.664002E-08
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1.999999999999997
            Iterations: 4
            Function evaluations: 17
            Gradient evaluations: 4

 *** Xopt in LSQ = (array([1.99999994, 0.99999995]), -1.999999999999997, 4, 0, 'Optimization terminated successfully.') 




  NIT    FC           OBJFUN            GNORM
    1     4     7.000000E+00     8.485281E+00
    2

For now, let's use the simpler BFGS (unconstrained) solver. Define a cost function denoting the distance from the robot end-effector to an arbitrary target. 

In [12]:
# %load tp1/solution_optimize_q.py
import numpy as np
from scipy.optimize import fmin_bfgs


def placement(x, y, theta):
    return [y, 0, x, 0, np.sin(theta / 2), 0, np.cos(theta / 2)]


def display2d(q):
    '''Display the robot in Gepetto Viewer. '''
    assert (q.shape == (2, 1))
    c0 = np.cos(q[0, 0])
    s0 = np.sin(q[0, 0])
    c1 = np.cos(q[0, 0] + q[1, 0])
    s1 = np.sin(q[0, 0] + q[1, 0])
    gv.applyConfiguration('world/joint1', placement(0, 0, 0))  # noqa
    gv.applyConfiguration('world/arm1', placement(c0 / 2, s0 / 2, q[0, 0]))  # noqa
    gv.applyConfiguration('world/joint2', placement(c0, s0, q[0, 0]))  # noqa
    gv.applyConfiguration('world/arm2', placement(c0 + c1 / 2, s0 + s1 / 2, q[0, 0] + q[1, 0]))  # noqa
    gv.applyConfiguration('world/joint3', placement(c0 + c1, s0 + s1, q[0, 0] + q[1, 0]))  # noqa
    gv.refresh()  # noqa


def endeffector(q):
    '''Return the 2D position of the end effector of the robot at configuration q. '''
    assert (q.shape == (2, 1))
    c0 = np.cos(q[0, 0])
    s0 = np.sin(q[0, 0])
    c1 = np.cos(q[0, 0] + q[1, 0])
    s1 = np.sin(q[0, 0] + q[1, 0])
    return np.matrix([c0 + c1, s0 + s1]).T


target = np.matrix([.5, .5]).T


def cost(q):
    q = np.matrix(q).T
    eff = endeffector(q)
    return np.linalg.norm(eff - target)


def callback(q):
    q = np.matrix(q).T
    display2d(q)
    import time
    time.sleep(.1)


x0 = np.array([0.0, 0.0])
xopt_bfgs = fmin_bfgs(cost, x0, callback=callback)
print('\n *** Xopt in BFGS = %s \n\n\n\n' % xopt_bfgs)


         Current function value: 0.000000
         Iterations: 30
         Function evaluations: 304
         Gradient evaluations: 73

 *** Xopt in BFGS = [1.99482736 3.8643269 ] 






## What configuration to optimize?
It seems logical to optimize over the angles $q_1,q_2$. However, other representations of the configuration are possible. Consider for example the explicit representation, where the placement of each body 1,2,3 is stored. For each body, we get $x,y,\theta$, so 9 parameters in total. In addition, each body position is constrained with respect to the placement of the previous body, with 6 constraints in total. 

What are the pros and cons? The effector position is now a trivial function of the representation, hence the cost function is very simple. The trade-off is that we have to explicitly satisfy the constraints. 

Let's start by defining the configuration.

In [13]:
x1, y1, th1, x2, y2, th2, x3, y3, th3 = x = np.zeros(9)

The cost function is now just a sparse difference on x3,y3:

In [14]:
target = [.5, .5]

def cost_9(x):
    x1, y1, th1, x2, y2, th2, x3, y3, th3 = x 
    return (x3 - target[0]) ** 2 + (y3 - target[1]) ** 2

The constraint function should return a vector, each coefficient corresponding to one of the 6 constraints:

In [15]:
def constraint_9(x):
    x1, y1, th1, x2, y2, th2, x3, y3, th3 = x 
    from numpy import cos, sin
    return np.array([
        x1 - 0,
        y1 - 0,
        x1 + cos(th1) - x2,
        y1 + sin(th1) - y2,
        x2 + cos(th2) - x3,
        y2 + sin(th2) - y3,
    ])

In [16]:
x0 = np.zeros(9)
print(cost_9(x0), constraint_9(x0))

0.5 [0. 0. 1. 0. 1. 0.]


The callback function is now accepting dimension 9 vector:

In [17]:
def callback_9(ps):
    '''Display the robot in Gepetto Viewer. '''
    assert(ps.shape == (9,) )
    x1, y1, t1, x2, y2, t2, x3, y3, t3 = ps
    gv.applyConfiguration('world/joint1', placement(x1, y1, t1) )
    gv.applyConfiguration('world/arm1', placement(x1 + np.cos(t1) / 2, x1 + np.sin(t1) / 2, t1))
    gv.applyConfiguration('world/joint2', placement(x2, y2, t2) )
    gv.applyConfiguration('world/arm2', placement(x2 + np.cos(t2) / 2, y2 + np.sin(t2) / 2, t2))
    gv.applyConfiguration('world/joint3', placement(x3, y3, t3) )
    gv.refresh()
    import time
    time.sleep(.5)
    
callback_9(x0)

The BFGS solver defined above cannot be used directly to optimize over equality constraints. A dirty trick is to add the constraint as a penalty, i.e. a high-weigth term in the cost function: $penalty(x) = cost(x) + 100*||constraint(x)||^2$ . Here, we are in a good case where the optimum corresponds to the 0 of both the constraint and the cost. The penalty with any weight would lead to the optimum and perfect constraint satisfaction. Yet the solver suffers to reach the optimum, because of the way we have described the constraint.

In [18]:
# %load tp1/solution_optimize_placements_bfgs.py
# flake8: noqa


def penalty(x):
    return cost_9(x) + 100 * np.linalg.norm(constraint_9(x))**2


xopt_bfgs = fmin_bfgs(penalty, x0, callback=callback_9)
print('\n *** Xopt BFGS = %s \n\n\n\n' % xopt_bfgs)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 49
         Function evaluations: 693
         Gradient evaluations: 63

 *** Xopt BFGS = [-1.94847064e-08 -2.81093797e-08  1.99482960e+00 -4.11439880e-01
  9.11436868e-01 -4.24030954e-01  4.99997984e-01  4.99999127e-01
  0.00000000e+00] 






Alternatively, the solver S-LS-QP (sequential least-square quadratic-program) optimizes over equality constraints.

In [19]:
%load -r tp1/solution_optimize_placements.py


  NIT    FC           OBJFUN            GNORM
    1    11     5.000000E-01     1.414214E+00
    2    22     2.260000E+00     3.006659E+00
    3    33     1.745455E+00     2.642313E+00
    4    45     1.229163E+00     2.217352E+00
    5    56     2.636509E-01     1.026939E+00
    6    67     8.257470E-02     5.747163E-01
    7    78     1.490827E-02     2.441989E-01
    8    89     5.791089E-03     1.521984E-01
    9   100     5.748589E-05     1.516387E-02
   10   111     1.466559E-08     2.421909E-04
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.731271590554328e-11
            Iterations: 10
            Function evaluations: 112
            Gradient evaluations: 10

 *** Xopt SQP  = [ 0.          0.         -0.42403634  0.91143568 -0.41144267  1.99483082
  0.49999469  0.49999374  0.        ]






When properly defining the constraint, the solver converges quickly. It is difficult to say a-priori whether it is better to optimize with the q (and consequently a dense cost and no constraint) or with the x-y-theta (and consequently a sparse cost and constraints). Here, we empirically observe no difference. 